# Market generator Bühler - Yahoo Finance Data

In [1]:
import os
# Set working directory to MarketGenerators folder
# If you are working on LRZ servers, create the folder "MarketGenerators" and then specify something like
# Use "nvcr.io#nvidia/tensorflow:20.03-tf1-py3" container for cloud computing
path = "/dss/dsshome1/02/YOUR_LRZ_USER_NAME/MarketGenerators"
os.chdir(path)

In [ ]:
!pip install -r requirements_buehler.txt

In [3]:
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

import BuehlerVAE.src.market_generator as market_generator
from BuehlerVAE.src.utils import tosig
import BuehlerVAE.src.process_discriminator as process_discriminator

## Server computing part

In [ ]:
# Turn interactive plotting off
plt.ioff()
def run_buehler_pipeline(start, end, n_years_out):
    import BuehlerVAE.src.process_discriminator as process_discriminator
    ticker = "^GSPC"
    # Signature truncation order
    order = 4
    seed = 42
    method = "YFinance"
    freq = "M"
    n_out = n_years_out * 12 if freq == "M" else n_years_out
    params = {
        "S0": 1.,
        "ticker": "^GSPC",
        "start": start,
        "end": end
    }

    values = {k: v for k, v in params.items() if k not in ['S0', 'n', 'n_points', 'T']}.values()
    values_str = [str(value).replace('.', ',') for value in values]
    model_spec = '_'.join(values_str)
    name_string = f"_{method}_{model_spec}"
    MG = market_generator.MarketGenerator(ticker, freq=freq, sig_order=order, own_params=params, method=method, seed=seed)

    # Plot input paths and save to plots_buehler folder
    fig = plt.figure(figsize=(18, 10))
    plt.rcParams.update({'font.size': 22})
    for path in MG.windows:
        returns = path[::2, 1]
        plt.plot(returns, "b", alpha=0.1)

    plt.title("Paths")
    plt.xlabel("Days")
    plt.savefig(f"plots_buehler/input_paths{name_string}")
    plt.close(fig)

    # Train model
    MG.train(n_epochs=10000)

    # Generate new paths with same format
    params_out = params
    MG_out = market_generator.MarketGenerator(ticker, freq=freq, sig_order=order, own_params=params_out, method=method)

    # Generated
    normalised_generated = np.array([MG.generate(cond, normalised=True) for cond in MG_out.conditions])
    # Change 8 here if sig order changes! (Make flexible in future)
    generated = np.empty((0, 8))
    # First generate based on existing data
    for cond in MG_out.conditions[:-1]:
        generated = np.append(generated, MG.generate(cond).reshape(1, -1), axis=0)

    scaled_gen = MG_out.conditions[-1]
    for _ in range(n_out - MG_out.conditions.shape[0] + 1):
        scaled_gen = MG.generate(scaled_gen, normalised=True)
        generated = np.append(generated, MG.scaler.inverse_transform(scaled_gen.reshape(1, -1))[0].reshape(1, -1), axis=0)
    sigs1 = np.array([tosig.logsig2sig(logsig, 2, order) for logsig in tqdm(normalised_generated)])
    # New seed here, i.e. "out of sample" test
    MG._load_own_data(params, method)
    MG.orig_logsig = np.array([MG._logsig(path) for path in MG.windows])
    sigs2 = np.array([tosig.logsig2sig(logsig, 2, order) for logsig in tqdm(MG.scaler.transform(MG.orig_logsig[1:]))])

    test_stat = process_discriminator.T(set1=sigs1, set2=sigs2, order=order, compute_sigs=False, verbose=False)

    K = 1.

    with open(f"buehler_output_new/sigSummary{name_string}.txt", "w") as text_file:
        text_file.write(f'confidence level = {np.exp(-test_stat**2/16*len(sigs1))*100:.4f}%\n' +
                        f'confidence level (other metric) = {np.exp(-(np.sqrt(len(sigs1)/(2*K)*test_stat)-1)**2/2)*100:.4f}%'
                        )
    print(f"Saving file 'buehler_output_new/sigs_generated{name_string}.npy'...")
    np.save(f"buehler_output_new/sigs_generated{name_string}.npy", generated)
    return

In [ ]:
for start in [
    "1999-07-01", "2004-07-01", "2009-07-01", "2014-07-01",
    "2019-07-01", "2021-07-01", "2022-07-01"
]:
    for end in ["2024-06-30"]:
        for n_years_out in [1000]:
            run_buehler_pipeline(start, end, n_years_out)